In [1]:
import os
import shutil
import sys
from collections import Counter
from copy import deepcopy
from pathlib import Path

abs_module_path = Path("./../../").resolve()
if (abs_module_path.exists()) and (str(abs_module_path) not in sys.path):
    sys.path.append(str(abs_module_path)) # add path to scan customized module

from modules.data.processeddatainstance import ProcessedDataInstance
from modules.ml.utils import get_seg_desc
from modules.shared.config import load_config
from modules.shared.pathnavigator import PathNavigator
from modules.shared.utils import create_new_dir, exclude_paths, get_repo_root

In [ ]:
processed_di = ProcessedDataInstance()
processed_di.parse_config("ml_analysis.toml")

In [ ]:
config = load_config("ml_analysis.toml")
# [seg_results]
seg_desc = get_seg_desc(config)

seg_desc

In [ ]:
src_dir = processed_di.palmskin_processed_dir
seg_dirs = exclude_paths(list(src_dir.glob(f"*/{seg_desc}/*")), [".tif"])

seg_dirname_cnt = Counter()
for seg_dir in seg_dirs:
    seg_dirname = seg_dir.name
    seg_dirname_cnt.update([seg_dirname])

seg_dirname_cnt

In [ ]:
path_navigator = PathNavigator()
result_ml_dir = path_navigator.dbpp.get_one_of_dbpp_roots("result_ml")
collect_root = result_ml_dir.joinpath("Collected",
                                     processed_di.instance_name,
                                     seg_desc)

for seg_dirname in seg_dirname_cnt.keys():
    
    dst_dir = collect_root.joinpath(seg_dirname)
    create_new_dir(dst_dir)
    
    paths = list(src_dir.glob(f"*/{seg_desc}/{seg_dirname}/*.seg2o.png"))
    for path in paths:
        
        dname = path.relative_to(src_dir).parts[0]
        seg_suffix = path.suffixes[-2]
        
        # get feature
        path_split = deepcopy(list(path.parts))
        path_split[-1] = path_split[-1].replace(f"{seg_suffix}.png", ".ana.toml") # replace file ext.
        toml_file = Path(*path_split) # re-construct path
        cell_cnt = load_config(toml_file)["cell_count"]
        
        # copy and rename image
        new_path = dst_dir.joinpath(f"cnt{cell_cnt}_{dname}.png")
        print(new_path)
        shutil.copy(path, new_path)